In [1]:
import sys

modules_dir = 'D:/usr14/project/Binary/adiabaticbinary/ww'
sys.path.insert(0, modules_dir)

In [2]:
from modules import resnet1
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np
from readdata import read_dataset

In [3]:
device=torch.device('cuda')
def fit(model:resnet1.ResNet, optim,lossfunc,trainloader: DataLoader):
    model.train()
    totalloss = 0
    for data, target in trainloader:
        data, target = data.to(device), target.to(device)
        optim.zero_grad()
        output = model(data).to(device)
        loss = lossfunc(output, target)
        loss.backward()
        optim.step()
        with torch.no_grad():
            totalloss += loss.item()*data.size(0)
    return totalloss/len(trainloader.sampler)

def evaluate(model:resnet1.ResNet, val_loader: DataLoader,lossfunc:nn.CrossEntropyLoss):
    model.eval()
    loss=0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs).to(device)
            loss+=lossfunc(outputs,labels).item()*inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct+=(predicted==labels).sum().item()

    return loss/len(val_loader.sampler) ,correct / total

In [8]:
model = resnet1.ResNet(False, False).to(device)
lossfunc=nn.CrossEntropyLoss().to(device)
train_loader,val_loader,test_loader=read_dataset(64)

In [9]:
lr=0.1
counter=0
min_val_loss=np.inf
for epoch in range(150):
    if counter/10==1:
        counter=0
        lr*=0.5
        print(f"lr reduced to {lr}")
    optim=torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
    loss=fit(model,optim,lossfunc,train_loader)
    val_loss,val_acc=evaluate(model,val_loader,lossfunc)
    print(f'epoch: {epoch+1}, loss: {loss}, val_loss: {val_loss}, val_acc:{val_acc}')
    if val_loss<min_val_loss:
        min_val_loss=val_loss
        print(f"val loss reduced to {min_val_loss}")
        counter=0
        torch.save(model.state_dict(), 'pre1.pth')
    else:
        counter+=1

OutOfMemoryError: CUDA out of memory. Tried to allocate 392.00 MiB. GPU 0 has a total capacity of 23.99 GiB of which 0 bytes is free. Of the allocated memory 22.84 GiB is allocated by PyTorch, and 40.96 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)